In [1]:
import os
HOME = os.getcwd()
print(f'HOME directory path: \'{HOME}\'')

HOME directory path: '/Users/constantin/Documents/Python_files/Machine_Learning/EY Challenge/Storm Damage - 2024/notebooks/google_colab'


In [2]:
import torch

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
import ultralytics
from ultralytics import YOLO
from IPython.display import Image

# Progress bar
from tqdm import tqdm

import numpy as np

# Others
import os
import shutil
import zipfile

%matplotlib inline

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
ultralytics.checks()

Ultralytics 8.3.163 🚀 Python-3.12.11 torch-2.7.1 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 32.0 GB RAM, 690.4/1858.2 GB disk)


In [4]:
model_name = 'yolo'
model_version = '11' # 'v8' or '11'
model_variant = 'n' # parameter required for model training phase


In [9]:
model = YOLO("../../trained_model/best_pre_trained/best.pt")  # load a custom trained model
model.info()

YOLO11n summary: 181 layers, 2,590,230 parameters, 0 gradients, 6.4 GFLOPs


(181, 2590230, 0, 6.4416768)

In [25]:
images_path = "../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/"
images_inference = [images_path+filename for filename in os.listdir(images_path)]
images_inference

['../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_15.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_4_68.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_18_83.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_37_60.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_30_69.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_3_61.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_43_87.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_6_2.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_3_75.jpg',
 '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_43_93.jpg',
 '../../dataset

In [40]:
images_inference_0_1000 = images_inference[:1000]
len(images_inference_0_1000)

1000

In [41]:
# list_images = ["/content/Pre_Event_tile_2_24.jpg", "/content/Pre_Event_tile_2_26.jpg"]
inference_results = model.predict(images_inference_0_1000, save=False, imgsz=640, conf=0.5)


0: 640x640 1 undamagedcommercialbuilding, 3 undamagedresidentialbuildings, 100.9ms
1: 640x640 (no detections), 100.9ms
2: 640x640 1 undamagedcommercialbuilding, 1 undamagedresidentialbuilding, 100.9ms
3: 640x640 1 undamagedcommercialbuilding, 21 undamagedresidentialbuildings, 100.9ms
4: 640x640 5 undamagedresidentialbuildings, 100.9ms
5: 640x640 8 undamagedresidentialbuildings, 100.9ms
6: 640x640 1 undamagedresidentialbuilding, 100.9ms
7: 640x640 (no detections), 100.9ms
8: 640x640 (no detections), 100.9ms
9: 640x640 (no detections), 100.9ms
10: 640x640 (no detections), 100.9ms
11: 640x640 1 undamagedresidentialbuilding, 100.9ms
12: 640x640 (no detections), 100.9ms
13: 640x640 (no detections), 100.9ms
14: 640x640 31 undamagedresidentialbuildings, 100.9ms
15: 640x640 (no detections), 100.9ms
16: 640x640 1 undamagedcommercialbuilding, 2 undamagedresidentialbuildings, 100.9ms
17: 640x640 (no detections), 100.9ms
18: 640x640 38 undamagedresidentialbuildings, 100.9ms
19: 640x640 (no detect

In [42]:
from collections import defaultdict

# images_containing_commercial_buildings : dict[str] = defaultdict(str)

images_containing_commercial_buildings: dict[str, str] = {}

temp = []

for i, inference_result in enumerate(inference_results):
  classes_ids =  {class_name: class_id for class_id, class_name in inference_result.names.items()}
  image_objects_detected = inference_result.boxes.cls.cpu().numpy()
  class_id = classes_ids['undamagedcommercialbuilding']
  if class_id in image_objects_detected:
    print(f'i: {i} - {inference_result.path} - {np.count_nonzero(image_objects_detected == class_id)}')
    temp.append(inference_result.path)


i: 0 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_15.jpg - 1
i: 2 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_18_83.jpg - 1
i: 3 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_37_60.jpg - 1
i: 16 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_29.jpg - 1
i: 25 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_42_8.jpg - 3
i: 27 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_24_34.jpg - 1
i: 30 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_39_65.jpg - 1
i: 36 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_43_44.jpg - 3
i: 42 - ../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_44_59.jpg - 2
i: 46 - ../../datasets/satellite_imagery/

In [43]:
len(temp)

170

In [44]:
import os
import shutil

In [45]:
def copy_files(images_paths: list[str], destination_dir: str):

  # Create a source directory
  os.makedirs(name=destination_dir, exist_ok=True)

  for image in images_paths:
    shutil.copy2(image, destination_dir)
    print(f'Copied image \'{image}\' to directory \'{destination_dir}\'.')

In [46]:
destination_dir = '../../datasets/dataset_creation/pre_event/batch_1'
images_paths = temp

copy_files(
    images_paths = images_paths,
    destination_dir = destination_dir
)

Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_15.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_18_83.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_37_60.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_29.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_42_8.jpg' to directory '../../datasets/dataset_creation/pre_event/batch_1'.
Copied image '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_24_34.jpg' to director

In [33]:
path_to_copy =  '../../datasets/dataset_creation/pre_event/batch_1'

In [36]:
image_1 = '../../datasets/satellite_imagery/maxar_high_resolution/pre_JPEG_Tiles/Pre_Event_tile_23_15.jpg'
shutil.copy2(image_1, dest_path)

'../../datasets/dataset_creation/pre_event/batch_1/Pre_Event_tile_23_15.jpg'